In [3]:
!pip install -q openai
!pip install langchain langchain-openai -q
!pip install faiss-cpu -q
!pip install langchain_community -q
!pip install tiktoken -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.5/362.5 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 997.8/997.8 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 391.5/391.5 kB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.4/140.4 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 44.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0

In [1]:
content = """
문서 1:
북극곰은 바다 얼음에 의존하여 물범을 사냥하기 때문에 기후 변화에 직접적인 영향을 받고 있습니다. 지구 온난화로 인해 바다 얼음이 매년 더 일찍 녹고 늦게 형성되면서,
북극곰이 사냥할 수 있는 시간이 줄어들고 있습니다. 이는 북극곰의 신체 상태 악화, 새끼의 생존율 저하, 그리고 일부 경우에는 사망률 증가로 이어졌습니다.

문서 2:
해빙 서식지의 상실은 북극곰 개체군에 가장 큰 위협 중 하나입니다. 얼음이 녹으면서 북극곰은 먹이를 찾기 위해 더 먼 거리를 이동해야 하며 이로 인해 에너지 소모가 증가합니다.
이로 인해 영양실조와 생식 성공률이 감소하여 종을 더욱 위험에 빠뜨릴 수 있습니다.

문서 3:
기후 변화는 북극곰의 사냥터뿐만 아니라 보금자리 구역에도 영향을 미치고 있습니다. 더 따뜻한 기온과 불안정한 눈 조건으로 인해
임신한 암컷이 적합한 보금자리 지역을 찾는 것이 더 어려워지고 있으며, 이는 새끼의 출생과 생존에 매우 중요합니다.
이는 바다 얼음 손실로 이미 어려움을 겪고 있는 북극곰 개체군에 추가적인 위험 요소가 되고 있습니다.

문서 4:
전통적인 식량 공급원이 점점 더 어려워지면서 일부 북극곰은 인간의 배설물을 먹거나 바다새와 그 알을 잡아먹는 모습이 관찰되었습니다.
이러한 행동은 단기적인 안도감을 제공할 수 있지만, 혹독한 북극 환경에서 장기적으로 생존하는 데 필수적인 물개로부터 얻는 고지방 식단을 대체하지는 못합니다.

문서 5:
최근 연구에 따르면 온실 가스 배출의 현재 추세가 계속된다면 북극곰은 이 세기 안에 멸종에 직면할 수 있습니다.
보존 노력은 전 세계 배출량을 줄이고 중요한 북극곰 서식지를 보호하는 데 집중하고 있지만 기후가 현재 속도로 계속 따뜻해진다면 이러한 노력만으로는 충분하지 않을 수 있습니다.
"""

with open('documents.txt', 'w') as f:
  f.write(content)

In [4]:
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
import os
os.environ['OPENAI_API_KEY'] = ''

loader = TextLoader('documents.txt')
documents = loader.load()

embeddings = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(documents, embeddings)

llm = ChatOpenAI(model_name="gpt-4o-mini-2024-07-18", temperature=0.3)

retriever = vectorstore.as_retriever(search_kwargs={'k':5})

qa_chain = RetrievalQA.from_chain_type(
    llm = llm,
    chain_type = "stuff",
    retriever = retriever,
    return_source_documents = True
)

query = '기후 변화가 북극곰에 어떤 영향을 미치는가?'
result = qa_chain.invoke({'query':query})

print(result['result'])

기후 변화는 북극곰에 여러 가지 부정적인 영향을 미치고 있습니다. 

1. **바다 얼음 감소**: 북극곰은 바다 얼음에 의존하여 물범을 사냥하는데, 지구 온난화로 인해 바다 얼음이 매년 더 일찍 녹고 늦게 형성되면서 사냥할 수 있는 시간이 줄어들고 있습니다. 이는 북극곰의 신체 상태 악화와 새끼의 생존율 저하로 이어질 수 있습니다.

2. **서식지 상실**: 해빙 서식지의 상실은 북극곰 개체군에 큰 위협이 되고 있습니다. 얼음이 녹으면서 북극곰은 먹이를 찾기 위해 더 먼 거리를 이동해야 하며, 이로 인해 에너지 소모가 증가하고 영양실조와 생식 성공률이 감소할 수 있습니다.

3. **보금자리 문제**: 기후 변화는 북극곰의 보금자리 구역에도 영향을 미치고 있습니다. 더 따뜻한 기온과 불안정한 눈 조건으로 인해 임신한 암컷이 적합한 보금자리 지역을 찾는 것이 더 어려워지고, 이는 새끼의 출생과 생존에 중요한 영향을 미칩니다.

4. **식량 공급의 변화**: 전통적인 식량 공급원이 줄어들면서 일부 북극곰은 인간의 배설물이나 바다새와 그 알을 잡아먹는 행동을 보이고 있습니다. 그러나 이러한 대체 식단은 고지방 식단을 제공하지 못해 장기적인 생존에 도움이 되지 않습니다.

5. **멸종 위험**: 최근 연구에 따르면 온실 가스 배출이 현재 추세로 계속된다면 북극곰은 이 세기 안에 멸종에 직면할 수 있습니다. 보존 노력에도 불구하고 기후가 현재 속도로 계속 따뜻해진다면 이러한 노력만으로는 충분하지 않을 수 있습니다.
